# Correlating model predictions with real signal values.

This notebook assesses model accuracy by comparing predicted to real signal values.

### Correlation Analysis: 
Measures linear (Pearson $r$) and rank-based (Spearman $\rho$) relationships. Uses Mean Squared Error (MSE) to measure prediction deviation. Organizes performance metrics by Species and Group (e.g., Test sets).

### Example output table 

| Species | Group | Metric | Model ID |
| :--- | :--- | :--- | :--- |
| Cow | Test | Pearson ($r$) | 0.422 |
| | | Pearson $p$-val | $1.88 \times 10^{-109}$ |
| | | Spearman ($\rho$) | 0.43 |
| | | Spearman $p$-val | $2.47 \times 10^{-114}$ |
| | | MSE | 2.52 |

In [22]:
import pandas as pd
import scipy.stats
import matplotlib.pyplot as plt
import numpy as np

# helper functions

def pearson_spearman(x, y):
    pearson_corr, pearson_p_value = scipy.stats.pearsonr(x, y)
    print(f"Pearson correlation coefficient: {pearson_corr:.4f}, p-value: {pearson_p_value:.4g}")

    spearman_corr, spearman_p_value = scipy.stats.spearmanr(x, y)
    print(f"Spearman correlation coefficient: {spearman_corr:.4f}, p-value: {spearman_p_value:.4g}")

species_list = ['macaque', 'rat', 'cow', 'pig']

def mean_squared_error(x, y):
    x = np.asarray(x)
    y = np.asarray(y)
    return np.mean((x - y) ** 2)

def format_value(metric_name, value):
    """Format values depending on whether it's a P-value metric or not."""
    # Check for '_p' which is more general for 'pearson_p', 'spearman_p' etc.
    if "_p" in metric_name:
        return f"{value:.2e}"  # Scientific notation for p-values
    else:
        return f"{value:.3g}"  # General format for other metrics



### Set global parameters



In [23]:
# Multiple hypothesis correction coefficient
MHC = 200

### Helper functions

`correlate` calculates the Pearson correlation coefficient ($r$), the Spearman rank correlation coefficient ($\rho$), and the Mean Squared Error (MSE) for the positive signal regions: training, validation, and test sets as well as the subsets (val2, val3, test2, and test3).

Goal: correlation coefficients ~ 1; MSE ~ 0

`negatives` calculates average prediction values (APV) for the inaccessible regions: training, validation, and test sets as well as the subsets (val1 and test1).

Goal: APV ~ 0

In [51]:
def correlate(mhc):
    rows = []
    # Lists for correlation calculations
    groups = ['Train', 'Validation', 'Test', 'Val2', 'Val3', 'Test2', 'Test3']
    preds = [pred_trainPos, pred_valPos, pred_testPos, pred_val2_df, pred_val3_df, pred_test2_df, pred_test3_df]
    trues = [doubled_trainPos, doubled_valPos, doubled_testPos, doubled_val2_df, doubled_val3_df, doubled_test2_df, doubled_test3_df]

    # Calculate correlations
    for group, pred_df, true_df in zip(groups, preds, trues):
        x = true_df.squeeze()
        y = pred_df.squeeze()
        pearson, pearson_p = scipy.stats.pearsonr(x, y)
        spearman, spearman_p = scipy.stats.spearmanr(x, y)
        mse = mean_squared_error(x, y)
        pearson_p *= mhc
        spearman_p *= mhc
        
        rows.append({'Group': group, 'Metric': 'Pearson', 'Value': pearson})
        rows.append({'Group': group, 'Metric': 'Pearson_p', 'Value': pearson_p})
        rows.append({'Group': group, 'Metric': 'Spearman', 'Value': spearman})
        rows.append({'Group': group, 'Metric': 'Spearman_p', 'Value': spearman_p})
        rows.append({'Group': group, 'Metric': 'MSE', 'Value': mse})
          
    return pd.DataFrame(rows)

def negatives():
    rows = []
     # Lists for negative average calculations
    negGroup = ['Train neg', 'Val neg', 'Test neg', 'Val1 avg pred', 'Test1 avg pred']
    negValues = [pred_trainNeg.mean().iloc[0], pred_valNeg.mean().iloc[0], pred_testNeg.mean().iloc[0], pred_val1_df.mean().iloc[0], pred_test1_df.mean().iloc[0]]
    
    # Add negative value averages
    for group, negv in zip(negGroup, negValues):
        rows.append({'Group': group, 'Metric': 'Avg Neg Prediction', 'Value': negv})

    return pd.DataFrame(rows)

### Declare the species and model lists for evaluation
Note that mouse is processed separately since it does not have the evaluation subsets.

In [52]:
# Example: 
species_list = ['macaque', 'rat', 'cow', 'pig']
model_list = ['bdbi7l3n'] # Can list any set of models that are trained on data with same processing strategy (ie. 500bp log-transofrmed signal)

### Code for non-mouse species

In [53]:
all_results = []
neg_results = []

for species in species_list:
    for model in model_list:
        model_dir = f'{model}_FINAL'

        # Load and process all dfs so correlate() function can access them
        #############################################################################

        # Load pos and neg TRAIN DFs

        # Update the negative training path based on the current species being processed.
        # These filenames follow a specific 'nonSpecies_andOthers' naming convention.
        negTrainPath = f'/home/azstephe/liverRegression/regression_liver/data/splits/negatives/nonRat_liver_andMacaque_andCow_andPig_TRAIN_500bp.bed'
        if species == 'macaque':
            negTrainPath = f'/home/azstephe/liverRegression/regression_liver/data/splits/negatives/nonMacaque_liver_andRat_andCow_andPig_TRAIN_500bp.bed'
        elif species == 'cow':
            negTrainPath = f'/home/azstephe/liverRegression/regression_liver/data/splits/negatives/nonCow_liver_andMacaque_andRat_andPig_TRAIN_500bp.bed'
        elif species == 'pig':
            negTrainPath = f'/home/azstephe/liverRegression/regression_liver/data/splits/negatives/nonPig_liver_andMacaque_andRat_andCow_TRAIN_500bp.bed'

        # Load the model's output (activations) for the entire training set (Pos + Neg).
        pred_TRAIN = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/model_outputs/{model_dir}/activations_{species}_TRAIN.csv', header=None)

        # Load metadata for the Positive and Negative training sets.
        # iloc[:,4] extracts the 5th column (signal value) for comparison.
        trainPos = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/splits/logPos/{species}_liver_TRAINONLY.narrowPeak', header=None, delim_whitespace=True).iloc[:,4]
        trainNeg = pd.read_csv(negTrainPath, header=None, delim_whitespace=True).iloc[:,4]

        trainPos_len = 2*len(trainPos)
        trainNeg_len = 2*len(trainNeg)
        
        if len(pred_TRAIN) != trainPos_len+trainNeg_len:
            print(f"ERROR TRAIN ({species}, {model}): predictions are a different length than validation sets")

        # The model generates predictions for both the original sequence and its Reverse Complement (RC).
        # To align the original metadata with these predictions, we duplicate each row and interleave 
        # them so that each sequence index appears twice in a row (e.g., Index 0, 0, 1, 1...).
        doubled_trainPos = pd.concat([trainPos, trainPos]).sort_index(kind='mergesort').reset_index(drop=True)
        doubled_trainNeg = pd.concat([trainNeg, trainNeg]).sort_index(kind='mergesort').reset_index(drop=True)

        # Extract the actual predictions from the consolidated model output (pred_TRAIN)
        # This assumes pred_TRAIN contains all positive predictions followed by all negative predictions.
        pred_trainPos = pred_TRAIN.head(trainPos_len)
        pred_trainNeg = pred_TRAIN.tail(trainNeg_len)

        #############################################################################

        # Load pos and neg VAL DFs

        negValPath = f'/home/azstephe/liverRegression/regression_liver/data/splits/negatives/nonRat_liver_andMacaque_andCow_andPig_VAL_500bp.bed'
        if species == 'macaque':
            negValPath = f'/home/azstephe/liverRegression/regression_liver/data/splits/negatives/nonMacaque_liver_andRat_andCow_andPig_VAL_500bp.bed'
        elif species == 'cow':
            negValPath = f'/home/azstephe/liverRegression/regression_liver/data/splits/negatives/nonCow_liver_andMacaque_andRat_andPig_VAL_500bp.bed'
        elif species == 'pig':
            negValPath = f'/home/azstephe/liverRegression/regression_liver/data/splits/negatives/nonPig_liver_andMacaque_andRat_andCow_VAL_500bp.bed'
        
        pred_VAL_ortho = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/model_outputs/{model_dir}/activations_{species}_VAL_orthologs.csv', header=None)
        
        valPos = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/splits/logPos/{species}_liver_VAL.narrowPeak', header=None, delim_whitespace=True).iloc[:,4]
        valNeg = pd.read_csv(negValPath, header=None, delim_whitespace=True).iloc[:,4]
        
        valPos_len = 2*len(valPos)
        valNeg_len = 2*len(valNeg)
        
        if len(pred_VAL_ortho) != valPos_len+valNeg_len:
            print(f"ERROR VALORTHO ({species}, {model}): predictions are a different length than validation sets")
        
        doubled_valPos = pd.concat([valPos, valPos]).sort_index(kind='mergesort').reset_index(drop=True)
        doubled_valNeg = pd.concat([valNeg, valNeg]).sort_index(kind='mergesort').reset_index(drop=True)
        
        pred_valPos = pred_VAL_ortho.head(valPos_len)
        pred_valNeg = pred_VAL_ortho.tail(valNeg_len)

        #############################################################################

        # Load pos and neg TEST DFs
        pred_TEST_ortho = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/model_outputs/{model_dir}/activations_{species}_TEST_orthologs.csv', header=None)
        
        testPos = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/test_splits/log_pos_LiuAll/{species}_liver_TEST_500bp.bed', header=None, delim_whitespace=True).iloc[:,4]
        testNeg = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/test_splits/neg/{species}_liver_TEST_500bp.bed', header=None, delim_whitespace=True).iloc[:,4]
        
        testPos_len = 2*len(testPos)
        testNeg_len = 2*len(testNeg)
        
        if len(pred_TEST_ortho) != testPos_len+testNeg_len:
            print(f"ERROR TEST ORTHO ({species}, {model}): predictions are a different length than validation sets")
        doubled_testPos = pd.concat([testPos, testPos]).sort_index(kind='mergesort').reset_index(drop=True)
        doubled_testNeg = pd.concat([testNeg, testNeg]).sort_index(kind='mergesort').reset_index(drop=True)
        
        pred_testPos = pred_TEST_ortho.head(testPos_len)
        
        pred_testNeg = pred_TEST_ortho.tail(testNeg_len)

        #############################################################################
        # Load VAL SET 1,2,3 DFs
        pred_VAL = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/model_outputs/{model_dir}/activations_{species}_VAL.csv', header=None)
        
        val1_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/val_splits/val1/{species}_liver_VAL_500bp.bed', header=None, delim_whitespace=True).iloc[:,4]
        val2_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/splits/log_val2/{species}_liver_VAL.narrowPeak', header=None, delim_whitespace=True).iloc[:,4]
        val3_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/splits/log_val3/{species}_liver_VAL.narrowPeak', header=None, delim_whitespace=True).iloc[:,4]
        
        val1_len = 2*len(val1_df)
        val2_len = 2*len(val2_df)
        val3_len = 2*len(val3_df)
        
        if len(pred_VAL) != val1_len+val2_len+val3_len:
            print(f"ERROR VAL ({species}, {model}): predictions are a different length than validation sets")
        
        doubled_val1_df = pd.concat([val1_df, val1_df]).sort_index(kind='mergesort').reset_index(drop=True)
        doubled_val2_df = pd.concat([val2_df, val2_df]).sort_index(kind='mergesort').reset_index(drop=True)
        doubled_val3_df = pd.concat([val3_df, val3_df]).sort_index(kind='mergesort').reset_index(drop=True)
        
        pred_val1_df = pred_VAL.head(val1_len)
        pred_val2_df = pred_VAL.iloc[val1_len:val1_len + val2_len]
        pred_val3_df = pred_VAL.tail(val3_len)

        #############################################################################
        # Load TEST SET 1,2,3  DFs
        pred_TEST = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/model_outputs/{model_dir}/activations_{species}_TEST.csv', header=None)
        
        test1_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/test_splits/log_LiuAll_test1/{species}_liver_TEST_500bp.bed', header=None, delim_whitespace=True).iloc[:,4]
        test2_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/test_splits/log_test2/{species}_liver_TEST_500bp.bed', header=None, delim_whitespace=True).iloc[:,4]
        test3_df = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/test_splits/log_test3/{species}_liver_TEST_500bp.bed', header=None, delim_whitespace=True).iloc[:,4]
        
        test1_len = 2*len(test1_df)
        test2_len = 2*len(test2_df)
        test3_len = 2*len(test3_df)
        
        if len(pred_TEST) != test1_len+test2_len+test3_len:
            print(f"ERROR TEST ({species}, {model}): predictions are a different length than validation sets")
        
        doubled_test1_df = pd.concat([test1_df, test1_df]).sort_index(kind='mergesort').reset_index(drop=True)
        doubled_test2_df = pd.concat([test2_df, test2_df]).sort_index(kind='mergesort').reset_index(drop=True)
        doubled_test3_df = pd.concat([test3_df, test3_df]).sort_index(kind='mergesort').reset_index(drop=True)
        
        pred_test1_df = pred_TEST.head(test1_len)
        pred_test2_df = pred_TEST.iloc[test1_len:test1_len + test2_len]
        pred_test3_df = pred_TEST.tail(test3_len)

        # Call the correlate function which now uses the globally available DFs
        corr_df = correlate(MHC)
        corr_df['species'] = species
        corr_df['model'] = model
        all_results.append(corr_df)
        
        neg_df = negatives()
        neg_df['species'] = species
        neg_df['model'] = model
        neg_results.append(neg_df)

# #############################################################################
# FINAL PROCESSING
# #############################################################################

summary_df = pd.concat(all_results)

summary_neg_df = pd.concat(neg_results)

# Define the custom order to place negative groups at the bottom.
custom_group_order = [
    'Train', 'Validation', 'Test', 'Val2', 'Val3', 'Test2', 'Test3'
]

custom_group_order_neg = [
    'Train neg', 'Val neg', 'Test neg', 'Val1 avg pred', 'Test1 avg pred'
]

# Convert 'group' to a categorical type with the specified order.
summary_df['Group'] = pd.Categorical(summary_df['Group'], categories=custom_group_order, ordered=True)
summary_neg_df['Group'] = pd.Categorical(summary_neg_df['Group'], categories=custom_group_order_neg, ordered=True)


# Pivot so each model is a column
pivot_df = summary_df.pivot_table(
    index=['species', 'Group', 'Metric'],
    columns='model',
    values='Value'
)

pivot_neg_df = summary_neg_df.pivot_table(
    index=['species', 'Group', 'Metric'],
    columns='model',
    values='Value'
)

# Sort the index to maintain custom order
pivot_df = pivot_df.sort_index(level=['species', 'Group', 'Metric'])
pivot_neg_df = pivot_neg_df.sort_index(level=['species', 'Group', 'Metric'])

pivot_df_reordered = pivot_df[model_list]
pivot_neg_df_reordered = pivot_neg_df[model_list]

metric_vars = ['Pearson', 'Pearson_p', 'Spearman', 'Spearman_p', 'MSE']
pivot_df_reordered = pivot_df_reordered.reindex(metric_vars, level='Metric')

# Loop through each model's column to apply the formatting
for col in pivot_df_reordered.columns:
    pivot_df_reordered[col] = pivot_df_reordered.apply(
        lambda row: format_value(row.name[2], row[col]),
        axis=1)

# display(pivot_df_reordered)

model                     bdbi7l3n
species Group Metric              
cow     Train Pearson        0.406
              Pearson_p   0.00e+00
              Spearman       0.401
              Spearman_p  0.00e+00
              MSE             2.27
...                            ...
rat     Test3 Pearson        0.317
              Pearson_p   6.76e-47
              Spearman       0.309
              Spearman_p  3.72e-44
              MSE              1.8

[140 rows x 1 columns]

### Helper functions for mouse

In [54]:
def correlate_mouse():
    rows = []
    # Lists for correlation calculations
    groups = ['Train', 'Validation', 'Test']
    preds = [pred_trainPos, pred_valPos, pred_testPos ]
    trues = [doubled_trainPos, doubled_valPos, doubled_testPos]

    # Calculate correlations
    for group, pred_df, true_df in zip(groups, preds, trues):
        x = true_df.squeeze()
        y = pred_df.squeeze()
        pearson, pearson_p = scipy.stats.pearsonr(x, y)
        spearman, spearman_p = scipy.stats.spearmanr(x, y)
        mse = mean_squared_error(x, y)
        pearson_p *= mhc
        spearman_p *= mhc
        rows.append({'Group': group, 'Metric': 'Pearson', 'Value': pearson})
        rows.append({'Group': group, 'Metric': 'Pearson_p', 'Value': pearson_p})
        rows.append({'Group': group, 'Metric': 'Spearman', 'Value': spearman})
        rows.append({'Group': group, 'Metric': 'Spearman_p', 'Value': spearman_p})
        rows.append({'Group': group, 'Metric': 'MSE', 'Value': mse})
    return pd.DataFrame(rows)

def negatives_mouse():
    rows = []
     # Lists for negative average calculations
    negGroup = ['Val neg', 'Test neg']
    negValues = [pred_valNeg.mean().iloc[0], pred_testNeg.mean().iloc[0]]
    
    # Add negative value averages
    for group, negv in zip(negGroup, negValues):
        rows.append({'Group': group, 'Metric': 'Avg Neg Prediction', 'Value': negv})

    return pd.DataFrame(rows)

In [58]:
all_results = []
neg_results = []
species_list = ['mouse']

for species in species_list:
    for model in model_list:
        model_dir = f'{model}_FINAL'
        
        # load TRAIN VAL DFs
        pred_TRAIN_VAL = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/model_outputs/{model_dir}/activations_{species}_TRAIN_VAL.csv', header=None)
        
        trainPos = pd.read_csv('/home/azstephe/liverRegression/regression_liver/data/splits/logPos/mouse_liver_TRAINONLY.narrowPeak', header=None, delim_whitespace=True).iloc[:,4]
        valPos = pd.read_csv('/home/azstephe/liverRegression/regression_liver/data/splits/logPos/mouse_liver_VAL.narrowPeak', header=None, delim_whitespace=True).iloc[:,4] 
        valNeg = pd.read_csv('/home/azstephe/regression_liver/data/splits/negatives/nonMouse_liver_andRat_andCow_andPig_andMacaque_VAL_500bp.bed', header=None, delim_whitespace=True).iloc[:,4]

        trainPos_len = 2*len(trainPos)
        valPos_len = 2*len(valPos)
        valNeg_len = 2*len(valNeg)
        
        if len(pred_TRAIN_VAL) != trainPos_len+valPos_len+valNeg_len:
            print(f"ERROR TRAIN ({species}, {model}): predictions are a different length than validation sets")
        
        doubled_trainPos = pd.concat([trainPos, trainPos]).sort_index(kind='mergesort').reset_index(drop=True)
        doubled_valPos = pd.concat([valPos, valPos]).sort_index(kind='mergesort').reset_index(drop=True)
        doubled_valNeg = pd.concat([valNeg, valNeg]).sort_index(kind='mergesort').reset_index(drop=True)
        
        pred_trainPos = pred_TRAIN_VAL.head(trainPos_len)
        pred_valPos = pred_TRAIN_VAL.iloc[trainPos_len:trainPos_len + valPos_len]
        pred_valNeg = pred_TRAIN_VAL.tail(valNeg_len)

        #############################################################################

        # load TEST ORTHO DFs
        pred_TEST_ortho = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/model_outputs/{model_dir}/activations_{species}_TEST.csv', header=None)
        
        testPos = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/test_splits/log_pos/mouse_liver_TEST_500bp.bed', header=None, delim_whitespace=True).iloc[:,4]
        testNeg = pd.read_csv(f'/home/azstephe/liverRegression/regression_liver/data/test_splits/neg/mouse_liver_TEST_500bp.bed', header=None, delim_whitespace=True).iloc[:,4]
        
        testPos_len = 2*len(testPos)
        testNeg_len = 2*len(testNeg)
        
        if len(pred_TEST_ortho) != testPos_len+testNeg_len:
            print(f"ERROR TEST ORTHO ({species}, {model}): predictions are a different length than validation sets")
        
        doubled_testPos = pd.concat([testPos, testPos]).sort_index(kind='mergesort').reset_index(drop=True)
        doubled_testNeg = pd.concat([testNeg, testNeg]).sort_index(kind='mergesort').reset_index(drop=True)
        
        pred_testPos = pred_TEST_ortho.head(testPos_len)
        pred_testNeg = pred_TEST_ortho.tail(testNeg_len)


        # Call the correlate function which now uses the globally available DFs
        corr_df = correlate_mouse()
        corr_df['species'] = species
        corr_df['model'] = model
        all_results.append(corr_df)
        
        neg_df = negatives_mouse()
        neg_df['species'] = species
        neg_df['model'] = model
        neg_results.append(neg_df)
        

# #############################################################################
# FINAL PROCESSING
# #############################################################################

summary_df = pd.concat(all_results)

summary_neg_df = pd.concat(neg_results)

# Define the custom order to place negative groups at the bottom.
custom_group_order = [
    'Train', 'Validation', 'Test'
]

custom_group_order_neg = [
    'Val neg', 'Test neg'
]

# Convert 'group' to a categorical type with the specified order.
summary_df['Group'] = pd.Categorical(summary_df['Group'], categories=custom_group_order, ordered=True)
summary_neg_df['Group'] = pd.Categorical(summary_neg_df['Group'], categories=custom_group_order_neg, ordered=True)


# Pivot so each model is a column
pivot_df = summary_df.pivot_table(
    index=['species', 'Group', 'Metric'],
    columns='model',
    values='Value'
)

pivot_neg_df = summary_neg_df.pivot_table(
    index=['species', 'Group', 'Metric'],
    columns='model',
    values='Value'
)

# Sort the index to maintain a logical order (will now use the custom group order)
pivot_df = pivot_df.sort_index(level=['species', 'Group', 'Metric'])
pivot_neg_df = pivot_neg_df.sort_index(level=['species', 'Group', 'Metric'])

pivot_df_reordered_mouse = pivot_df[model_list]
pivot_neg_df_reordered_mouse = pivot_neg_df[model_list]

metric_vars = ['Pearson', 'Pearson_p', 'Spearman', 'Spearman_p', 'MSE']
pivot_df_reordered_mouse = pivot_df_reordered_mouse.reindex(metric_vars, level='Metric')

# Loop through each model's column to apply the formatting
for col in pivot_df_reordered_mouse.columns:
    pivot_df_reordered_mouse[col] = pivot_df_reordered_mouse.apply(
        # Access the 'metric' from the index using row.name[2]
        # (assuming it's the 3rd level of your index)
        lambda row: format_value(row.name[2], row[col]),
        axis=1)

# display(pivot_df_reordered_mouse)

### Clean up final table

In [59]:
# concatenate mouse and the rest of the species into one dataframe
pos_predictions_eval_df = pd.concat([pivot_df_reordered, pivot_df_reordered_mouse])
neg_predictions_eval_df = pd.concat([pivot_neg_df_reordered, pivot_neg_df_reordered_mouse])

output_pos_filename = '~/data/tables/pos_log_model_eval_table.tsv'
# pos_predictions_eval_df.to_csv(output_pos_filename, sep='\t', float_format='%.3f')

output_neg_filename = '~/data/tables/neg_log_model_eval_table.tsv'
# neg_predictions_eval_df.to_csv(output_neg_filename, sep='\t', float_format='%.3f')

### Final Outputs:
`~/data/tables/pos_log_model_eval_table.tsv`

`~/data/tables/neg_log_model_eval_table.tsv`